<a href="https://colab.research.google.com/github/gregorywmorris/MLZoom2022/blob/main/Homework/MLZoomcampWeek3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Week 3 HW

Homework
Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.

Dataset
In this homework, we will use the California Housing Prices data from Kaggle.

Here's a wget-able link:

wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
We'll keep working with the 'median_house_value' variable, and we'll transform it to a classification task.


# Features
For the rest of the homework, you'll need to use only these columns:

* 'latitude',
* 'longitude',
* 'housing_median_age',
* 'total_rooms',
* 'total_bedrooms',
* 'population',
* 'households',
* 'median_income',
* 'median_house_value',
* 'ocean_proximity'

In [231]:
import pandas as pd 
import numpy as np 

import seaborn as sns
from matplotlib import pyplot as pyplot
%matplotlib inline

In [232]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
!wget $data -O data.csv 

--2022-09-26 15:45:49--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.36M  --.-KB/s    in 0.05s   

2022-09-26 15:45:50 (25.5 MB/s) - ‘data.csv’ saved [1423529/1423529]



In [233]:
df = pd.read_csv('data.csv') 

# Data preparation
Select only the features from above and fill in the missing values with 0.
Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.
Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.
Create a new column population_per_household by dividing the column population by the column households from dataframe.









In [234]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
  df[c] = df[c].str.lower().str.replace(' ', '_')

df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [235]:
df = df.copy()
df = df.fillna(0)
df.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [236]:
df['rooms_per_household'] = df.total_rooms / df.households
df['bedrooms_per_room'] = df.total_bedrooms / df.total_rooms
df['population_per_household'] =  df.population / df.households
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household'],
      dtype='object')

# Question 1
What is the most frequent observation (mode) for the column ocean_proximity?

<br>Options:


* **< 1H OCEAN**


In [237]:
df.ocean_proximity.mode()

0    <1h_ocean
dtype: object

# Split the data
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value (median_house_value) is not in your dataframe.

In [238]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [239]:
df['above_average'] = (df.median_house_value > df.median_house_value.mean())
df['above_average'] = (df['above_average']==True).astype(int)
df['above_average']

0        1
1        1
2        1
3        1
4        1
        ..
20635    0
20636    0
20637    0
20638    0
20639    0
Name: above_average, Length: 20640, dtype: int64

In [240]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [241]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_full_train = df_full_train.reset_index(drop=True)

In [242]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

In [243]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [244]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [245]:
len(df) == len(df_train)+len(df_val)+len(df_test)

True

In [246]:
df_full_train.above_average.value_counts(normalize=True) # sanity check

0    0.592236
1    0.407764
Name: above_average, dtype: float64

# Question 2
Create the correlation matrix for the numerical features of your train dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
What are the two features that have the biggest correlation in this dataset?

<br>Options:

* **total_bedrooms and households**


In [247]:
corr_matrix = df.corr().abs() # absolute value, we want to see the biggest correlation, direction doesn't matter for this question.

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

corr_values = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) #<- without .astype(bool) you get an error'Boolean array expected for the condition, not float64'
                  .stack() # Formats the data as you see below
                  .sort_values(ascending=False) # sorts highest to lowest.
                  .head()) 

#first element of sol series is the pair with the biggest correlation
corr_values


total_bedrooms  households        0.966507
longitude       latitude          0.924664
total_rooms     total_bedrooms    0.920196
                households        0.918484
population      households        0.907222
dtype: float64

In [248]:
corr_matrix # this is why we use the .triu, notice if we get everying from the upper triangle we don't get duplicates. 

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,bedrooms_per_room,population_per_household,above_average
longitude,1.000000,0.924664,0.108197,0.044568,0.068082,0.099773,0.055310,0.015176,0.045967,0.027540,0.084836,0.002476,0.072633
latitude,0.924664,1.000000,0.011173,0.036100,0.065318,0.108785,0.071035,0.079809,0.144160,0.106389,0.104112,0.002366,0.098810
housing_median_age,0.108197,0.011173,1.000000,0.361262,0.317063,0.296244,0.302916,0.119034,0.105623,0.153277,0.125396,0.013191,0.080069
total_rooms,0.044568,0.036100,0.361262,1.000000,0.920196,0.857126,0.918484,0.198050,0.134153,0.133798,0.174583,0.024581,0.128782
total_bedrooms,0.068082,0.065318,0.317063,0.920196,1.000000,0.866266,0.966507,0.007295,0.049148,0.002717,0.122205,0.028019,0.056209
population,0.099773,0.108785,0.296244,0.857126,0.866266,1.000000,0.907222,0.004834,0.024650,0.072213,0.031397,0.069863,0.014898
households,0.055310,0.071035,0.302916,0.918484,0.966507,0.907222,1.000000,0.013033,0.065843,0.080598,0.059818,0.027309,0.074778
median_income,0.015176,0.079809,0.119034,0.198050,0.007295,0.004834,0.013033,1.000000,0.688075,0.326895,0.573836,0.018766,0.541778
median_house_value,0.045967,0.144160,0.105623,0.134153,0.049148,0.024650,0.065843,0.688075,1.000000,0.151948,0.238759,0.023737,0.804353
rooms_per_household,0.027540,0.106389,0.153277,0.133798,0.002717,0.072213,0.080598,0.326895,0.151948,1.000000,0.387465,0.004852,0.112104


# Make median_house_value binary
* We need to turn the median_house_value variable from numeric into binary.

* Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.


In [249]:
above_average = (df_full_train.median_house_value > df_full_train.median_house_value.mean())
above_average = (above_average==True).astype(int)
above_average

0        0
1        1
2        0
3        0
4        0
        ..
16507    1
16508    0
16509    1
16510    1
16511    1
Name: median_house_value, Length: 16512, dtype: int64

# Question 3
Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
What is the value of mutual information?
Round it to 2 decimal digits using round(score, 2)

<br>Options:


* **0.10**


In [250]:
from sklearn.metrics import mutual_info_score 

In [251]:
round(mutual_info_score(df_full_train.ocean_proximity, df_full_train.above_average),2)

0.1

In [252]:
round(mutual_info_score(df_full_train.ocean_proximity, above_average),2)

0.1

# Question 4
* Now let's train a logistic regression
* Remember that we have one categorical variable ocean_proximity in the data; include it using one-hot encoding.
* Fit the model on the **training dataset**.
* To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the **validation dataset** and round it to 2 decimal digits.

<br>Options:


* **0.84**


In [253]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [254]:
dv = DictVectorizer(sparse = False)
train_dict = df_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dict)

In [255]:
val_dict = df_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

In [256]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(x_train, y_train) # long run due to 'df['above_average'] = (df.median_house_value > df.median_house_value.mean())' not being run in the beginning.

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [257]:
model.intercept_[0], model.coef_[0].round(3)

(-0.2467698702849388,
 array([ 0.73 ,  0.004,  0.036,  0.109,  0.084,  1.219,  0.408, -1.716,
         0.076,  0.226,  0.76 , -0.002,  0.01 , -0.016,  0.002, -0.   ]))

In [258]:
y_pred = model.predict_proba(x_val)[:, 1]

In [259]:
mean_val = (y_pred >= 0.5)

In [260]:
(y_val == mean_val).mean()

0.8355135658914729

# Question 5
* Let's find the least useful feature using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

<br> Which of following feature has the smallest difference?

* **households**



In [261]:
from sklearn.metrics import mutual_info_score

In [262]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household', 'above_average'],
      dtype='object')

In [263]:
features = ['total_rooms','total_bedrooms', 'population', 'households']

In [264]:
def mutual_info_feature_score(series):
  return mutual_info_score(series, df_full_train.above_average)

In [265]:
mi = df_full_train[features].apply(mutual_info_feature_score)
mi.sort_values(ascending = False) # this tells us which features should be the most important. 


total_rooms       0.228645
population        0.140274
total_bedrooms    0.071673
households        0.065052
dtype: float64

In [266]:
dv = DictVectorizer(sparse = False)
train_dict = df_train[features].to_dict(orient = 'records')
x_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(x_train, y_train) 

model.intercept_[0], model.coef_[0].round(3)

y_pred = model.predict_proba(x_val)[:, 1]

mean_val = (y_pred >= 0.5)

display((y_val == mean_val).mean())

0.7095445736434108

In [267]:
for i in features:
  f_train = df_train.copy()
  f_train = f_train[features]
  del f_train[i]

  dv = DictVectorizer(sparse = False)
  train_dict = f_train.to_dict(orient = 'records')
  x_train = dv.fit_transform(train_dict)

  val_dict = df_val.to_dict(orient='records')
  x_val = dv.transform(val_dict)

  model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
  model.fit(x_train, y_train) 

  model.intercept_[0], model.coef_[0].round(3)

  y_pred = model.predict_proba(x_val)[:, 1]

  mean_val = (y_pred >= 0.5)

  display((y_val == mean_val).mean())


0.6276647286821705

0.6608527131782945

0.656734496124031

0.6719961240310077

In [268]:
accuracy = np.array([0.6276647286821705,
0.6608527131782945,
0.656734496124031,
0.6719961240310077])

In [269]:
(accuracy - 0.8355135658914729)

array([-0.20784884, -0.17466085, -0.17877907, -0.16351744])

# Question 6
* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.
* This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
* If there are multiple options, select the smallest alpha.

<br> Options:

* **0**

In [270]:
from sklearn.linear_model import Ridge

In [271]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [272]:
def rmse(y, y_pred):
  error = y - y_pred
  se = error ** 2
  mse = se.mean()
  return np.sqrt(mse)

In [273]:
dv = DictVectorizer(sparse = False)
train_dict = df_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

In [276]:
for r in [0, 0.01, 0.1,1,10]:
      
    model_ridge = Ridge(alpha=r, solver="sag", random_state=42)
    model_ridge.fit(x_train, y_train)
    
    w0 = model_ridge.intercept_
    w = model_ridge.coef_
    y_pred = w0 + x_val.dot(w)
    
    score = rmse(y_val, y_pred)

    print(r, w0, round(score, 3))
    print()

0 11.95574814929651 0.524

0.01 11.955748149419218 0.524

0.1 11.955748150585048 0.524

1 11.95574816230478 0.524

10 11.955748279318072 0.524



# Submit the results
Submit your results here: https://forms.gle/vQXAnQDeqA81HSu86
You can submit your solution multiple times. In this case, only the last submission will be used
<br>If your answer doesn't match options exactly, select the closest one
<br>
#Deadline
The deadline for submitting is 26 September (Monday), 23:00 CEST.

After that, the form will be closed